<a href="https://colab.research.google.com/github/mjmousavi97/Computer-Vision/blob/main/projects/pro-005/src/squeezNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, datasets, models
from torch.utils.data import DataLoader
from torchsummary import summary
import wandb

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# Transforms for training and validation
data_transform = {
    'train': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),

    'val': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

In [4]:
wandb.init(
    project="SqueesNet-5flowers", config={
    "learning_rate": 0.001,
    "epochs": 50,
    "batch_size": 16,
    "architecture": "SqueesNet",
    "dataset": "5flowers",
    "pretrained": True,
    "optimizer": "Adam",
    "input_size": 224
    }
)

config = wandb.config

wandb: Currently logged in as: mohammadjavad-mousavi97 (mohammadjavad-mousavi97-arak-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [5]:
train_dir = "/content/drive/MyDrive/Colab Notebooks/flowers/train"
val_dir = "/content/drive/MyDrive/Colab Notebooks/flowers/val"

train_dataset = datasets.ImageFolder(train_dir, transform=data_transform['train'])
val_dataset = datasets.ImageFolder(val_dir, transform=data_transform['val'])

train_loader = DataLoader(train_dataset, batch_size=config.batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=config.batch_size, shuffle=False)

In [6]:
from torchvision.models import SqueezeNet1_1_Weights

model = models.squeezenet1_1(weights=SqueezeNet1_1_Weights.DEFAULT)
model.classifier[1] = nn.Conv2d(in_channels=512, out_channels=5, kernel_size=(1, 1))

for param in model.parameters():
    param.requires_grad = False

for param in model.classifier[1].parameters():
    param.requires_grad = True

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

wandb.watch(model, log="all", log_freq=10)

In [7]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=config.learning_rate)

In [8]:
def train_model(model, criterion, optimizer, train_loader, val_loader, num_epochs=10, log_every=10):
    device = next(model.parameters()).device

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0

        for step, (images, labels) in enumerate(train_loader):
            images = images.to(device, non_blocking=True)
            labels = labels.to(device, non_blocking=True)

            optimizer.zero_grad(set_to_none=True)
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            bs = labels.size(0)
            running_loss += loss.item() * bs
            preds = outputs.argmax(dim=1)
            correct += (preds == labels).sum().item()
            total += bs

            if (step + 1) % log_every == 0:
                train_loss_step = running_loss / max(1, total)
                train_acc_step = correct / max(1, total)
                wandb.log(
                    {
                        "epoch": epoch + 1,
                        "step": epoch * len(train_loader) + (step + 1),
                        "train/loss": train_loss_step,
                        "train/acc": train_acc_step,
                        "lr": optimizer.param_groups[0]["lr"],
                    }
                )

        train_loss = running_loss / max(1, total)
        train_acc = correct / max(1, total)

        model.eval()
        val_loss_sum = 0.0
        val_correct = 0
        val_total = 0

        with torch.no_grad():
            for images, labels in val_loader:
                images = images.to(device, non_blocking=True)
                labels = labels.to(device, non_blocking=True)

                outputs = model(images)
                loss = criterion(outputs, labels)

                bs = labels.size(0)
                val_loss_sum += loss.item() * bs
                preds = outputs.argmax(dim=1)
                val_correct += (preds == labels).sum().item()
                val_total += bs

        val_loss = val_loss_sum / max(1, val_total)
        val_acc = val_correct / max(1, val_total)

        wandb.log(
            {
                "epoch": epoch + 1,
                "train/loss_epoch": train_loss,
                "train/acc_epoch": train_acc,
                "val/loss": val_loss,
                "val/acc": val_acc,
            }
        )

        print(
            f"Epoch {epoch+1}/{num_epochs} | "
            f"train_loss={train_loss:.4f} train_acc={train_acc:.4f} | "
            f"val_loss={val_loss:.4f} val_acc={val_acc:.4f}"
        )

    print("Training complete!")

In [9]:
train_model(model, criterion, optimizer, train_loader, val_loader, num_epochs=config.epochs)

Epoch 1/50 | train_loss=0.6609 train_acc=0.7532 | val_loss=0.4298 val_acc=0.8497
Epoch 2/50 | train_loss=0.3815 train_acc=0.8660 | val_loss=0.4004 val_acc=0.8605
Epoch 3/50 | train_loss=0.3397 train_acc=0.8787 | val_loss=0.3629 val_acc=0.8773
Epoch 4/50 | train_loss=0.2956 train_acc=0.8945 | val_loss=0.4145 val_acc=0.8477
Epoch 5/50 | train_loss=0.2721 train_acc=0.9059 | val_loss=0.3829 val_acc=0.8764
Epoch 6/50 | train_loss=0.2547 train_acc=0.9109 | val_loss=0.3276 val_acc=0.8872
Epoch 7/50 | train_loss=0.2421 train_acc=0.9149 | val_loss=0.3347 val_acc=0.8942
Epoch 8/50 | train_loss=0.2414 train_acc=0.9092 | val_loss=0.3525 val_acc=0.8823
Epoch 9/50 | train_loss=0.2292 train_acc=0.9182 | val_loss=0.3234 val_acc=0.8971
Epoch 10/50 | train_loss=0.2130 train_acc=0.9232 | val_loss=0.3399 val_acc=0.8843
Epoch 11/50 | train_loss=0.2073 train_acc=0.9299 | val_loss=0.3047 val_acc=0.9060
Epoch 12/50 | train_loss=0.2087 train_acc=0.9217 | val_loss=0.3052 val_acc=0.9120
Epoch 13/50 | train_loss=